---
title: Tokenization Strategies
date: 2023-02-14
image: tokenization.jpg
categories: ["NLP"]
---

In [1]:
# | echo: false
%load_ext lab_black

<!-- ![](tokenization.jpg) -->
<img src="tokenization.jpg" alt="Imad" height="400px" width="600px">

## Introduction

**Tokenization** is the process of breaking down a string into smaller units of information that will be used by the model. This process sometimes involves some preprocessing steps such as converting to lowercase, [stemming & lemmatization](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html), etc. There are many tokenizations strategies that each has its own advantages and drawbacks. We will first consider the two extreme tokenization strategies: *character tokenization* and *word tokenization*. Then we will discuss subword tokenizations where statistical methods and language heuristics are used to learn the optimal splitting of words 

## Character Tokenization

This is the simplest tokenization strategy where we simply break down the text at the character level. Then the characters will be fed to the model. Consider the following example:

In [2]:
text = "I love NLP!"
list(text)

['I', ' ', 'l', 'o', 'v', 'e', ' ', 'N', 'L', 'P', '!']

From here, it is easy to convert each character into integers that would be fed to the model. This step is called *numericalization*. We can numericalize the above text by first building the vocabulary, and then convert each character to its corresponding index as follows:

In [3]:
vocab = {char: idx for idx, char in enumerate(sorted(list(text)))}
print(vocab)

{' ': 1, '!': 2, 'I': 3, 'L': 4, 'N': 5, 'P': 6, 'e': 7, 'l': 8, 'o': 9, 'v': 10}


Now we can simply map each token (character in this case) to its own corresponding index:

In [4]:
[vocab[char] for char in text]

[3, 1, 8, 9, 10, 7, 1, 5, 4, 6, 2]

- **Advantages**:
    - Helps us avoid misspellings and rare words
    - Very small vocabulary
- **Drawbacks**:
    - Sequences length will be very long
    - Linguistic structures such as words now need to be learned from data. This requires much more data, memory, and computation

## Word Tokenization

The other extreme of word tokenization is to split text into words and then map each word to its corresponding index in the vocabulary. The simplest form would be to split on whitespaces (which work well for English but not other languages such as Japanes that don't have a well-defined idea of a word):

In [5]:
text.split()

['I', 'love', 'NLP!']

In [6]:
vocab = {char: idx for idx, char in enumerate(sorted(text.split()))}
print(vocab)

{'I': 0, 'NLP!': 1, 'love': 2}


In [8]:
[vocab[word] for word in text.split()]

[0, 2, 1]

Most tokenizers would include rules and heauristics that try to separate parts of meaning even when there are no spaces such as "doesn't" into "does n't".

- **Advantages**:
    - Sequences length will be short
- **Drawbacks**:
    - Size of the vocabulary will explode for large corpus due to the fact that words can include declinations, misspellings, or punctuations. If the vocabulary size has 1m words and the embedding dimension is 512 -> the first embedding layer would be ~ 0.5 billion parameters!
        - We can work around this issue by includtion top n most frequent words. For example, if we include top 100,000 words -> the first embedding layer would be ~ 0.5 million parameters. However, because all other words will be mapped to the `UNK` token, the model has no idea about the words associated with the `UNK` token and we may lose some important information
    - Some languages don't have well-defined idea of what constitute a word

## Subword Tokenization

Split words inro smaller parts based on the most frequent sub-strings. Therefore, we want to split keep the most frequent words as unique entities but split the rare words into smaller units to allow us to deal with misspellings and complex words. This will help us achieve the best of both wolds: 1) manageable vocabulary size, 2) keep frequent words as their own entities, and 3) deal with complex and misspelling words.

The subword tokenizers are typically learned from pretraining corpus using statistical rules and algorithms. Let's illustrate an example from the tokenizer used by the [DistilBERT](https://arxiv.org/abs/1910.01108) model.

In [14]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
encoded_text = tokenizer(text)
encoded_text

{'input_ids': [101, 1045, 2293, 17953, 2361, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [16]:
tokenizer.convert_ids_to_tokens(encoded_text["input_ids"])

['[CLS]', 'i', 'love', 'nl', '##p', '!', '[SEP]']

Let's explain the output of the DistilBERT tokenizer:

- [CLS] is a special token that is used to indixate the start of a sequence
- [SEP] is also a special token to separate multiple sequences
- `##` prefix indicates that the previous string isn't white space
    - This shows that nlp is not common token, so it was split into two tokens
- We can also see that `!` has its own token

We can reconstruct that encoded text as follows:

In [17]:
tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(encoded_text["input_ids"])
)

'[CLS] i love nlp ! [SEP]'

## Conclusion

Through this post, we covered three tokenization strategies along with their advantages and challenges/limitations. We mostly use tokenizers from well-known libraries such as [spaCy](https://spacy.io/) because it is very hard to get it right ourselves.

When using pretrained models such as DistilBERT, we must use the same tokenizer that the model used during training. Otherwise, what the model assumes token_id = 1 is will be completely different that what the new token_id = 1 represents. It has the same effect as shuffling the vocabulary.